In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/covid-toy-dataset/covid_toy.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().any()

Here we need <br>
fever - Missing Values <br>
gender, city - Nominal Categorial Data <br>
cough - Ordinal Categorical Data <br>
has_covid - Label Encoder <br>

<p>If we do this manually one by one, its very hectic <br>
Column Trnasformer makes our life easier</p>

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

<h2>Without Column Transformer</h2>

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

In [ ]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

In [ ]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

In [ ]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

In [ ]:
X_train_final_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_final_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

<h2>Using Column Transformer!!</h2>

In [ ]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('imp',SimpleImputer(),['fever']),
    ('ode',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('ohe',OneHotEncoder(sparse=False,drop='first'),['gender','city']),
],remainder='passthrough')

X_train_coltrans = transformer.fit_transform(X_train)
X_test_coltrans = transformer.transform(X_test)

In [ ]:
print(X_train_coltrans.shape)
print(X_train_final_transformed.shape)
print(X_test_coltrans.shape)
print(X_test_final_transformed.shape)

In [ ]:
print(X_train_coltrans[0])
print(X_train_final_transformed[0])